In [2]:
"""多线程的基本使用

threading库可以在单独的线程中执行任何可调用的对象
创建好的线程不会立即执行，除非调用它的start()方法
"""

from threading import Thread
import time

# 目标函数
def countdown(n):
    while n > 0:
        print('Countdown: ', n)
        n -= 1
        time.sleep(1)
        
# 启动线程
t = Thread(target=countdown, args=(10,))
t.start()

# 判断线程的执行状态
if t.is_alive():
    print('Running...')
else:
    print('Completed!')
    
# 后台执行需要长时间运行的线程
# t = Thread(target=countdown, args=(10,), daemon=True)

Countdown: Running...
 10
Countdown:  9
Countdown:  8
Countdown:  7
Countdown:  6
Countdown:  5
Countdown:  4
Countdown:  3
Countdown:  2
Countdown:  1
